In [2]:
!pip install flask-ngrok

In [3]:
%cd /content/

/content


In [4]:
from flask import Flask,render_template,request
from flask_ngrok import run_with_ngrok
import torch

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!git init 

Initialized empty Git repository in /content/.git/


In [8]:
!git clone https://github.com/penda-diagne/CoursPytorch/

Cloning into 'CoursPytorch'...
remote: Enumerating objects: 566, done.
remote: Counting objects: 100% (566/566), done.
remote: Compressing objects: 100% (557/557), done.
remote: Total 566 (delta 356), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (566/566), 109.02 KiB | 2.66 MiB/s, done.
Resolving deltas: 100% (356/356), done.


In [9]:
%cd CoursPytorch

/content/CoursPytorch


In [10]:
%cd Projet_mboup/

/content/CoursPytorch/Projet_mboup


In [11]:
!pip install pytorch-lightning
!pip install wandb

     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 56.2 MB/s 
     |████████████████████████████████| 829 kB 48.5 MB/s 
     |████████████████████████████████| 132 kB 62.8 MB/s 
     |████████████████████████████████| 329 kB 26.7 MB/s 
     |████████████████████████████████| 1.1 MB 64.1 MB/s 
     |████████████████████████████████| 271 kB 66.9 MB/s 
     |████████████████████████████████| 160 kB 69.5 MB/s 
     |████████████████████████████████| 192 kB 58.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=ae16f7533c9093ce51b2398fea081f1fcfaf821d229fe09a36579846775a4940
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found exis

In [12]:
import fibrose_model

In [13]:
import torchvision.models as models
model = fibrose_model.LitModel((3,329,375), 2,"resnet")
model.load_state_dict(torch.load('/content/drive/MyDrive/models/weights.pt',map_location='cpu'),strict=False)
model.eval()



Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


LitModel(
  (accuracy): Accuracy()
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        

In [14]:
%cd /content/drive/MyDrive/projetFaari

/content/drive/MyDrive/projetFaari


In [15]:
from PIL import Image
import torchvision.transforms as transforms
from PIL import ImageFilter

def predict(path):
      augmentation = transforms.Compose([
              transforms.RandomHorizontalFlip(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
      transform = transforms.Compose([
              transforms.Resize(size=(329, 375)),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
      im=Image.open(path)
      im = im.filter(ImageFilter.SMOOTH)
      im = transform(im)
      # im = augmentation(im)
      reshaped = im.unsqueeze(0)
      pred=model(reshaped)
      prediction = int(torch.max(pred.data, 1)[1].numpy())
      return prediction

In [16]:
pred=predict('/content/drive/MyDrive/f4.9.1.JPG')
print(pred)

1


/content/CoursPytorch/Projet_mboup/fibrose_model.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(self.classifier(x))


In [ ]:
app = Flask(__name__,template_folder='templates')
run_with_ngrok(app)   
@app.route("/",methods=["GET","POST"])
def main():
  return render_template("index.html")
@app.route("/about")
def about_page():
  return "please subscribe AI hub"
@app.route("/submit",methods=["GET","POST"])

def get_submit():
  p=1
  img_path=""
  if request.method =='POST':
    img=request.files['my-image']
    img_path='static/'+img.filename
    img.save(img_path)    
    p=predict(img)
  return render_template("index.html",prediction=p,img_path=img_path)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4712-104-196-16-99.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Nov/2021 11:15:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 11:15:13] "GET /favicon.ico HTTP/1.1" 404 -
/content/CoursPytorch/Projet_mboup/fibrose_model.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(self.classifier(x))
127.0.0.1 - - [09/Nov/2021 11:51:18] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 11:51:19] "GET /static/f4.9.1.JPG HTTP/1.1" 200 -
